# Exercícios - Pandas 2.0

O objetivo desse notebook é ajudar na fixação dos conteúdos da aula de Pandas 2.0.

Caso haja alguma dúvida, consulte o material e as referências bibliográficas, mas também estamos à disposição por e-mail, slack ou telegram! Não hesite em falar conosco =D

## Setup

In [1]:
# importe a biblioteca
import pandas as pd

In [2]:
# leia os datasets utilizados na aula
cups = pd.read_csv('WorldCups.csv')
matches = pd.read_csv('WorldCupMatches.csv')
players = pd.read_csv('WorldCupPlayers.csv')

## Tipos de Dados em Tabelas

### Strings

In [3]:
# na tabela de jogadores, crie uma nova coluna com o sobrenome de cada jogador
players['Player Last Name'] = players['Player Name'].str.split(' ').str[-1].str.title()
players.head(3)

,RoundID,MatchID,Team Initials,Coach Name,Line-up,Shirt Number,Player Name,Position,Event,Player Last Name
0,201,1096,FRA,CAUDRON Raoul (FRA),S,0,Alex THEPOT,GK,NaN,Thepot
1,201,1096,MEX,LUQUE Juan (MEX),S,0,Oscar BONFIGLIO,GK,NaN,Bonfiglio
2,201,1096,FRA,CAUDRON Raoul (FRA),S,0,Marcel LANGILLER,NaN,G40',Langiller


In [4]:
# nessa mesma tabela, crie uma nova coluna com o país de origem do treinador
players['Coach County'] = players['Coach Name'].str.split(' ').str[-1]
players['Coach County'] = players['Coach County'].str.replace('(', '')
players['Coach County'] = players['Coach County'].str.replace(')', '')
players['Coach County'] = players['Coach County'].str.upper()
players.head(3)

,RoundID,MatchID,Team Initials,Coach Name,Line-up,Shirt Number,Player Name,Position,Event,Player Last Name,Coach County
0,201,1096,FRA,CAUDRON Raoul (FRA),S,0,Alex THEPOT,GK,NaN,Thepot,FRA
1,201,1096,MEX,LUQUE Juan (MEX),S,0,Oscar BONFIGLIO,GK,NaN,Bonfiglio,MEX
2,201,1096,FRA,CAUDRON Raoul (FRA),S,0,Marcel LANGILLER,NaN,G40',Langiller,FRA


### Datetime

In [5]:
matches['Datetime'] = matches['Datetime'].str.replace('June', 'Jun').str.replace('July', 'Jul')
matches['Datetime'] = pd.to_datetime(matches['Datetime'], format = '%d %b %Y - %H:%M ')

In [6]:
# assumindo que um jogo dura 2h, crie uma nova coluna na tabela de partidas com o horário do final de cada partida
matches['Datetime Final'] = matches['Datetime'].apply(lambda x: x + pd.DateOffset(hours=2))

In [7]:
import datetime as dt

In [8]:
dt.datetime.today()

datetime.datetime(2020, 6, 11, 22, 4, 49, 341614)

In [9]:
# crie uma nova coluna na tabela de partidas com a diferença de tempo entre o dia de hoje e o dia do jogo
matches['Datetime Difference'] = matches['Datetime'].apply(lambda x: dt.datetime.today() - x)

## Query

In [10]:
# quantas partidas tiveram um público de mais de 30000 pessoas?
matches.query('`Attendance` > 30000').shape

(636, 22)

In [11]:
# quantos jogos tiveram uma diferença de gols maior do que 5?
#matches['H', ]
matches.query('`Home Team Goals` - `Away Team Goals` > 5 or\
               `Home Team Goals` - `Away Team Goals` < 5 ').shape

(835, 22)

In [12]:
# quantas partidas ocorreram nas copas nas quais o Brasil foi campeão?
cups.query('`Winner` == "Brazil"').shape

(5, 10)

In [13]:
# quantos técnicos treinam a seleção de um país diferente do seu país de origem?
players.query('`Coach County` != `Team Initials`')\
    .drop_duplicates(['Coach County', 'Team Initials', 'Coach Name']).shape

(117, 11)

## Merge

In [14]:
# crie uma tabela única juntando as três tabelas atuais (partidas, copas e jogadores)
cups_matches = pd.merge(matches, cups, on='Year')
cups_matches_players = pd.merge(players, cups_matches, on='MatchID')
cups_matches_players.shape

(39256, 41)

## Group By

In [15]:
# quantas copas cada seleção ganhou?
cups.groupby(['Winner'])['Year'].count()

Winner
Argentina     2
Brazil        5
England       1
France        1
Germany       1
Germany FR    3
Italy         4
Spain         1
Uruguay       2
Name: Year, dtype: int64

In [16]:
cups = cups.dropna()

In [17]:
# qual o público médio de cada copa?
cups_matches.groupby('Year')['Attendance_x'].mean()

Year
1930.0    32808.277778
1934.0    21352.941176
1938.0    20872.222222
1950.0    47511.181818
1954.0    29561.807692
1958.0    23423.142857
1962.0    27911.625000
1966.0    48847.968750
1970.0    50124.218750
1974.0    49098.763158
1978.0    40678.710526
1982.0    40571.596154
1986.0    46039.057692
1990.0    48388.750000
1994.0    68991.115385
1998.0    43517.187500
2002.0    42268.703125
2006.0    52491.234375
2010.0    49669.625000
2014.0    55374.910256
Name: Attendance_x, dtype: float64

In [18]:
# quantos jogos cada jogador já jogou? E quantas copas?
players.groupby('Player Name')['MatchID'].count()

Player Name
?URI?I?        3
A BAUTISTA     4
A COLE        14
A GUARDADO     4
A MEDINA       4
              ..
�IGI?          3
�STENSTAD      4
�ULER          3
�UNJI?         3
�ZIL          18
Name: MatchID, Length: 7663, dtype: int64

In [19]:
# quantos jogos cada juiz apitou?
matches.groupby('Referee')['MatchID'].nunique()

Referee
ABD EL FATAH Essam (EGY)           1
ADAIR John (NIR)                   1
AGNOLIN Luigi (ITA)                4
AGUILAR ELIZALDE Abel (MEX)        1
AGUILAR Joel (SLV)                 2
                                  ..
WYSSLING Paul (SUI)                4
Wolfgang STARK (GER)               3
YAMASAKI MALDONADO Arturo (MEX)    5
ZECEVIC Konstantin (YUG)           1
ZSOLT Istvan (HUN)                 5
Name: MatchID, Length: 366, dtype: int64